In [4]:
#!/usr/bin/env python3
"""
data_collect.py

This script:
  1. Scrapes station data from CAIC (Basin, Beck, Boss) and saves them as text files.
  2. For the Beck station, fetches USGS NWIS surface temperature data directly from the URL,
     builds an interpolation function relative to the earliest time in beck_collect.txt,
     and overwrites beck_collect.txt with an extra column containing the USGS data.
"""

import requests
from bs4 import BeautifulSoup
import datetime
import numpy as np
import shutil
from scipy.interpolate import interp1d

###############################################################################
# SECTION 1: CAIC DATA SCRAPING
###############################################################################

# Station options with parameters for URL construction and output formatting.
station_options = {
    "Basin": {
        "station_title": "A-Basin+SA-Pali+(A-BasinSA)+11920+ft",
        "station_code": "CAABP",
        "expected_numeric_cols": 13,
        "remove_index": None
    },
    "Beck": {
        "station_title": "Senator+Beck+%28CSAS%29+12186+ft",
        "station_code": "CASBK",
        "expected_numeric_cols": 14,
        "remove_index": 8
    },
    "Boss": {
        "station_title": "Boss+Basin+%28USGS%29+11259+ft",
        "station_code": "USBBN",
        "expected_numeric_cols": 14,
        "remove_index": 7
    }
}

def scrape_station_data(end_date_str="2025-03-01+24", hours_range=72,
                        station_title="A-Basin+SA-Pali+(A-BasinSA)+11920+ft",
                        station_code="CAABP",
                        expected_numeric_cols=13,
                        remove_index=None):
    """
    Scrape CAIC station data for the given time range.
    Returns a list of strings. Each line is formatted as:
      YYYY Mon DD HH:MM  Temp MxTp MnTp Dew(F) RH Spd Dir Gst SWIN SWOUT LWIN LWOUT NET
    (17 columns total: 4 date/time columns plus 13 numeric values).
    """
    url = (
        "https://stations.avalanche.state.co.us/tabular.php"
        f"?title={station_title}"
        f"&st={station_code}"
        f"&date={end_date_str}"
        "&unit=e"
        "&area=caic"
        f"&range={hours_range}"
    )
    response = requests.get(url)
    if response.status_code != 200:
        raise RuntimeError(f"Request failed with status {response.status_code}")

    soup = BeautifulSoup(response.text, "html.parser")
    pre_tag = soup.find("pre")
    if not pre_tag:
        raise ValueError("Could not find <pre> block with data.")

    lines = pre_tag.text.strip().split("\n")

    def convert_to_24h(timestr_12h, ampm):
        """Convert a 12-hour time string (with am/pm) into 24-hour format."""
        hour_12, minute = timestr_12h.split(":")
        hour_12 = int(hour_12)
        minute = int(minute)
        if hour_12 == 12 and ampm.lower() == "am":
            return "00:00"
        if hour_12 == 12 and ampm.lower() == "pm":
            return f"12:{minute:02d}"
        if ampm.lower() == "pm":
            return f"{hour_12 + 12:02d}:{minute:02d}"
        return f"{hour_12:02d}:{minute:02d}"

    data_lines = []
    for row in lines:
        row = row.strip()
        # Skip headers and blank lines.
        if not row or row.startswith("Date") or row.startswith(station_title.replace("+", " ")):
            continue
        parts = row.split()
        if len(parts) < 5 + expected_numeric_cols:
            continue

        # Determine if an AM/PM token is present.
        possible_ampm = parts[4].lower()
        if possible_ampm in ("am", "pm"):
            time_12h = parts[3]
            ampm = parts[4]
            offset = 5
        else:
            time_12h = parts[3]
            ampm = None
            offset = 4

        year_str, month_str, day_str = parts[0], parts[1], parts[2]
        time_24 = convert_to_24h(time_12h, ampm) if ampm else time_12h

        # Extract numeric tokens.
        numeric_parts = parts[offset:]
        if len(numeric_parts) < expected_numeric_cols:
            continue
        numeric_cols = numeric_parts[:expected_numeric_cols]

        if remove_index is not None:
            del numeric_cols[remove_index]

        out_line = f"{year_str} {month_str} {day_str} {time_24}"
        for val in numeric_cols:
            out_line += f"  {val}"
        data_lines.append(out_line)

    return data_lines

def build_caic_url(end_date_str="2025-03-01+24", hours_range=72,
                   station_title="A-Basin+SA-Pali+(A-BasinSA)+11920+ft",
                   station_code="CAABP"):
    """Build the CAIC URL for tabular data."""
    return (
        "https://stations.avalanche.state.co.us/tabular.php"
        f"?title={station_title}"
        f"&st={station_code}"
        f"&date={end_date_str}"
        "&unit=e"
        "&area=caic"
        f"&range={hours_range}"
    )

def build_caic_plot_url(end_date_str="2025-03-01+24", hours_range=72,
                        station_title="A-Basin+SA-Pali+(A-BasinSA)+11920+ft",
                        station_code="CAABP"):
    """Build the CAIC URL for plotting data."""
    return (
        "https://stations.avalanche.state.co.us/hplot.php"
        f"?title={station_title}"
        f"&st={station_code}"
        f"&date={end_date_str}"
        "&unit=e"
        "&area=caic"
        f"&range={hours_range}"
    )

###############################################################################
# SECTION 2: USGS NWIS MERGE FUNCTIONS (for Senator Beck)
###############################################################################

def read_earliest_time_from_beck(beck_file):
    """
    Reads beck_collect.txt and returns the earliest timestamp (as datetime)
    from the first valid data row.
    """
    month_map = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    earliest_time = None
    with open(beck_file, 'r') as fin:
        for line in fin:
            line_str = line.strip()
            if not line_str:
                continue
            parts = line_str.split()
            if len(parts) < 17:
                continue
            try:
                year_str, month_str, day_str = parts[0], parts[1], parts[2]
                hhmm_str = parts[3]
                year = int(year_str)
                month = month_map[month_str]
                day = int(day_str)
                hh_part, mm_part = hhmm_str.split(':')
                hour_24 = int(hh_part)
                minute = int(mm_part)
                if hour_24 == 24:
                    hour_24 = 0
                    dt_obj = datetime.datetime(year, month, day) + datetime.timedelta(days=1)
                else:
                    dt_obj = datetime.datetime(year, month, day, hour_24, minute)
                earliest_time = dt_obj
                break
            except Exception:
                continue
    if earliest_time is None:
        raise ValueError("No valid data lines found in beck_collect.txt for earliest time.")
    return earliest_time

def fetch_usgs_data(usgs_url, earliest_time):
    """
    Fetches USGS NWIS data from the provided URL (in rdb format), parses it,
    and returns two numpy arrays: times (in hours since earliest_time) and temperatures (°C).
    
    Expected format (tab-delimited, with comment lines starting with '#' or header 'agency_cd'):
      USGS   375429107433201   2024-04-03 21:00   MDT   -11.0   P
    """
    response = requests.get(usgs_url)
    if response.status_code != 200:
        raise RuntimeError("Failed to fetch USGS data; status: " + str(response.status_code))
    text = response.text

    times_list = []
    temps_list = []
    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith('#') or line.startswith('agency_cd'):
            continue
        parts = line.split('\t')
        if len(parts) < 6:
            continue
        date_str = parts[2]   # e.g., "2024-04-03 21:00"
        temp_str = parts[4]   # e.g., "-11.0"
        try:
            # Try parsing with "%Y-%m-%d %H:%M" and fall back to "%Y-%m-%d %H:%M:%S" if needed.
            try:
                dt_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M")
            except ValueError:
                dt_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            continue
        try:
            T_c = float(temp_str)
        except ValueError:
            continue
        dt_diff = dt_obj - earliest_time
        dt_hours = dt_diff.total_seconds() / 3600.0
        times_list.append(dt_hours)
        temps_list.append(T_c)
    if len(times_list) == 0:
        raise ValueError("No valid USGS data found from URL.")
    times_arr = np.array(times_list)
    temps_arr = np.array(temps_list)
    idx_sort = np.argsort(times_arr)
    return times_arr[idx_sort], temps_arr[idx_sort]

def overwrite_beck_with_usgs(beck_file, usgs_temp_interp, earliest_time):
    """
    Reads beck_collect.txt line by line, re-parses the timestamp for each data row,
    computes the USGS temperature using the interpolation function, and appends
    the temperature (°C) as a new column. Overwrites beck_collect.txt in place,
    keeping a backup as beck_collect.txt.backup.
    """
    backup_file = beck_file + ".backup"
    shutil.copy2(beck_file, backup_file)

    month_map = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }

    with open(backup_file, 'r') as fin, open(beck_file, 'w') as fout:
        for line in fin:
            line_str = line.strip()
            if not line_str:
                fout.write(line)
                continue
            parts = line_str.split()
            if len(parts) < 17:
                fout.write(line)
                continue
            try:
                year_str, month_str, day_str = parts[0], parts[1], parts[2]
                hhmm_str = parts[3]
                year = int(year_str)
                month = month_map[month_str]
                day = int(day_str)
                hh_part, mm_part = hhmm_str.split(':')
                hour_24 = int(hh_part)
                minute = int(mm_part)
                if hour_24 == 24:
                    hour_24 = 0
                    dt_obj = datetime.datetime(year, month, day) + datetime.timedelta(days=1)
                else:
                    dt_obj = datetime.datetime(year, month, day, hour_24, minute)
                dt_hours = (dt_obj - earliest_time).total_seconds() / 3600.0
                usgs_T_c = usgs_temp_interp(dt_hours)
                new_line = f"{line_str}  {usgs_T_c:.2f}\n"
                fout.write(new_line)
            except Exception:
                fout.write(line)
    print(f"Overwrote {beck_file} with USGS Tsurf(C) appended. Backup saved as {backup_file}.")

###############################################################################
# SECTION 3: MAIN SCRIPT FLOW
###############################################################################

if __name__ == "__main__":
    # --- Step 1: Scrape CAIC data for all stations ---
    end_date_str = "2025-01-20+24"  # e.g., midnight leading into the next day
    hours_range = 24 * 10           # number of hours of data to fetch

    for station_key, station_info in station_options.items():
        print("Selected station:", station_key)
        caic_url = build_caic_url(
            end_date_str,
            hours_range,
            station_title=station_info["station_title"],
            station_code=station_info["station_code"]
        )
        caic_plot_url = build_caic_plot_url(
            end_date_str,
            hours_range,
            station_title=station_info["station_title"],
            station_code=station_info["station_code"]
        )
        print("URL being fetched:\n", caic_url)
        print("Plot URL being fetched:\n", caic_plot_url)

        data_lines = scrape_station_data(
            end_date_str,
            hours_range,
            station_title=station_info["station_title"],
            station_code=station_info["station_code"],
            expected_numeric_cols=station_info["expected_numeric_cols"],
            remove_index=station_info["remove_index"]
        )
        print(f"Scraped {len(data_lines)} data rows for station {station_key}.")

        out_filename = f"{station_key.lower()}_collect.txt"
        with open(out_filename, "w") as f:
            for line in data_lines:
                f.write(line + "\n")
        print(f"Wrote {len(data_lines)} lines to {out_filename}.\n")

    # --- Step 2: Merge USGS NWIS data into beck_collect.txt ---
    # This step is only for the Beck station.
    beck_file = "beck_collect.txt"
    try:
        # Get the earliest timestamp from beck_collect.txt
        earliest_time = read_earliest_time_from_beck(beck_file)
        # Use the provided USGS NWIS URL
        usgs_url = ("https://nwis.waterservices.usgs.gov/nwis/iv/?sites=375429107433201"
                    "&agencyCd=USGS&startDT=2024-04-03T20:15:56.246-06:00"
                    "&endDT=2025-04-03T20:15:56.246-06:00&parameterCd=72405&format=rdb")
        times_usgs, temps_usgsC = fetch_usgs_data(usgs_url, earliest_time)
        usgs_temp_interp = interp1d(times_usgs, temps_usgsC, kind='linear', fill_value='extrapolate')
        # Overwrite beck_collect.txt with the merged USGS surface temperature column.
        overwrite_beck_with_usgs(beck_file, usgs_temp_interp, earliest_time)
    except Exception as e:
        print("Skipping USGS merging for Beck. Reason:", e)

    print("Data collection complete.")


Selected station: Basin
URL being fetched:
 https://stations.avalanche.state.co.us/tabular.php?title=A-Basin+SA-Pali+(A-BasinSA)+11920+ft&st=CAABP&date=2025-01-20+24&unit=e&area=caic&range=240
Plot URL being fetched:
 https://stations.avalanche.state.co.us/hplot.php?title=A-Basin+SA-Pali+(A-BasinSA)+11920+ft&st=CAABP&date=2025-01-20+24&unit=e&area=caic&range=240
Scraped 241 data rows for station Basin.
Wrote 241 lines to basin_collect.txt.

Selected station: Beck
URL being fetched:
 https://stations.avalanche.state.co.us/tabular.php?title=Senator+Beck+%28CSAS%29+12186+ft&st=CASBK&date=2025-01-20+24&unit=e&area=caic&range=240
Plot URL being fetched:
 https://stations.avalanche.state.co.us/hplot.php?title=Senator+Beck+%28CSAS%29+12186+ft&st=CASBK&date=2025-01-20+24&unit=e&area=caic&range=240
Scraped 241 data rows for station Beck.
Wrote 241 lines to beck_collect.txt.

Selected station: Boss
URL being fetched:
 https://stations.avalanche.state.co.us/tabular.php?title=Boss+Basin+%28USGS%29